## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

if not os.path.exists("jet.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/jet.bmp --no-check-certificate

In [ ]:
jet = cv2.imread('jet.bmp', cv2.IMREAD_GRAYSCALE)

plt.imshow(jet)
plt.gray()
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
jet_hist = cv2.calcHist([jet], [0], None, [256], [0, 256])

lm = int(np.mean(jet))

H1 = jet_hist[0:lm]
H2 = jet_hist[lm:256]

C1 = H1.cumsum()/np.max(H1)
C2 = H2.cumsum()/np.max(H2)

C1 = C1*lm
C2 = lm + 1 + (255 - lm - 1)*C2

C1_normalized = cv2.normalize(C1, None, 0, lm, cv2.NORM_MINMAX)
C2_normalized = cv2.normalize(C2, None, lm, 255, cv2.NORM_MINMAX)

C = np.append(C1_normalized, C2_normalized)

jet_lut = cv2.LUT(jet, C)

jet_equalized = cv2.equalizeHist(jet)

plt.figure(figsize=(20, 6))

ax = plt.subplot(1, 3, 1)
ax.imshow(jet)
plt.xticks([]), plt.yticks([])
plt.title(label='Oryginalny obraz')

ax = plt.subplot(1, 3, 2)
ax.imshow(jet_lut)
plt.xticks([]), plt.yticks([])
plt.title(label='BBHE')

ax = plt.subplot(1, 3, 3)
ax.imshow(jet_equalized)
plt.gray()
plt.xticks([]), plt.yticks([])
plt.title(label='Wbudowana funkcja equalizeHist()')

plt.show()

# ---------------------------------------------

plt.figure(figsize=(20, 6))

ax = plt.subplot(1, 3, 1)
ax.plot(jet_hist)
plt.title(label='Oryginalny obraz')
plt.grid()

ax = plt.subplot(1, 3, 2)
ax.plot(cv2.calcHist([jet_lut], [0], None, [256], [0,256]))
plt.title(label='BBHE')
plt.grid()

ax = plt.subplot(1, 3, 3)
ax.plot(cv2.calcHist([jet_equalized], [0], None, [256], [0,256]))
plt.title(label='Wbudowana funkcja equalizeHist()')
plt.grid()
plt.show()

In [ ]:
amount_of_pixels = jet.shape[0]*jet.shape[1]

jet_hist_cum = jet_hist.cumsum()
tab = np.abs(jet_hist_cum - amount_of_pixels/2)
mean_value = int(np.min(tab))
lm = 0
while mean_value > jet_hist_cum[lm]:
    lm += 1

H1 = jet_hist[0:lm]
H2 = jet_hist[lm:256]

C1 = H1.cumsum()/np.max(H1)
C2 = H2.cumsum()/np.max(H2)

C1 = C1*lm
C2 = lm + 1 + (255 - lm - 1)*C2

C1_normalized = cv2.normalize(C1, None, 0, lm, cv2.NORM_MINMAX)
C2_normalized = cv2.normalize(C2, None, lm, 255, cv2.NORM_MINMAX)

C = np.append(C1_normalized, C2_normalized)

jet_lut = cv2.LUT(jet, C)

jet_equalized = cv2.equalizeHist(jet)

plt.figure(figsize=(20, 6))

ax = plt.subplot(1, 3, 1)
ax.imshow(jet)
plt.xticks([]), plt.yticks([])
plt.title(label='Oryginalny obraz')

ax = plt.subplot(1, 3, 2)
ax.imshow(jet_lut)
plt.xticks([]), plt.yticks([])
plt.title(label='DSIHE')

ax = plt.subplot(1, 3, 3)
ax.imshow(jet_equalized)
plt.gray()
plt.xticks([]), plt.yticks([])
plt.title(label='Wbudowana funkcja equalizeHist()')
plt.show()

# --------------------------------------------------------

plt.figure(figsize=(20, 6))

ax = plt.subplot(1, 3, 1)
ax.plot(jet_hist)
plt.title(label='Oryginalny obraz')
plt.grid()

ax = plt.subplot(1, 3, 2)
ax.plot(cv2.calcHist([jet_lut], [0], None, [256], [0,256]))
plt.title(label='DSIHE')
plt.grid()

ax = plt.subplot(1, 3, 3)
ax.plot(cv2.calcHist([jet_equalized], [0], None, [256], [0,256]))
plt.title(label='Wbudowana funkcja equalizeHist()')
plt.grid()
plt.show()